In [ ]:
import os

# Temporarily set the Hugging Face token (only for this runtime session)
os.environ["HUGGINGFACE_TOKEN"] = " "


In [29]:
import torch

In [3]:
import os
hf_token = os.getenv("HUGGINGFACE_TOKEN")
if not hf_token:
    raise EnvironmentError("❌ HUGGINGFACE_TOKEN not found. Set it using os.environ['HUGGINGFACE_TOKEN'] = 'your_token_here'")


In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM

#deepseek-ai/deepseek-llm-7b-chat
#deepseek-ai/deepseek-coder-6.7b-instruct
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-llm-7b-chat", use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-llm-7b-chat", use_auth_token=hf_token)


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:934: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [19]:
paragraph = """Remote work improves employee productivity. People working remotely often report fewer distractions and better focus.
EVs have zero tailpipe emissions. Most EVs are charged with electricity generated from fossil fuels.
AI-generated images mimic style but lack original thought. 3D-printed organs could reduce transplant wait times."""


In [43]:
import re

def split_into_sentences(text):
    # Replace line breaks with spaces and split on sentence-ending punctuation
    text = text.replace("\n", " ").strip()
    return re.split(r'(?<=[.!?])\s+', text)


In [44]:
sentences = split_into_sentences(paragraph)

for s in sentences:
    print("-", s)

- Remote work improves employee productivity.
- People working remotely often report fewer distractions and better focus.
- EVs have zero tailpipe emissions.
- Most EVs are charged with electricity generated from fossil fuels.
- AI-generated images mimic style but lack original thought.
- 3D-printed organs could reduce transplant wait times.


In [47]:
classified = []

for s in sentences:
    prompt = f"You are an assistant for argument mining. Classify the following sentence as either a 'claim' or a 'premise':\n\"{s}\"\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10,
            pad_token_id=tokenizer.eos_token_id
        )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    label = "claim" if "claim" in result.lower() else "premise"
    classified.append({"text": s, "type": label})


In [48]:
structured_output = {
    "claims": [{"text": c["text"]} for c in classified if c["type"] == "claim"],
    "premises": [{"text": c["text"]} for c in classified if c["type"] == "premise"]
}

# Step 5: View result
print(json.dumps(structured_output, indent=2))

{
  "claims": [
    {
      "text": "Remote work improves employee productivity."
    },
    {
      "text": "People working remotely often report fewer distractions and better focus."
    },
    {
      "text": "EVs have zero tailpipe emissions."
    },
    {
      "text": "Most EVs are charged with electricity generated from fossil fuels."
    },
    {
      "text": "AI-generated images mimic style but lack original thought."
    },
    {
      "text": "3D-printed organs could reduce transplant wait times."
    }
  ],
  "premises": []
}
